# CLASSIFICATION: UNCLASSIFIED

## Background 
Python has a few central structures for storing data - lists, sets, and dicts - and there are other common ones in external libraries - such as numpy.array and numpy.matrix. None of these are ideal for analyzing tabular data, in which we have rows with columns of different types (e.g. numerical, date, text, category), much like tables in database. This could be accomplished with the builtin types, but it would be very cumbersome to do some types of analysis.

Pandas is a package that allows you to represent tabular data as a DataFrame and perform many useful operations on it, such as:
* subsetting rows and columns based on complex criteria
* grouping and aggregating
* joining 
* creating new columns as functions of existing ones
* sorting by multiple columns

Pandas will allow you do anything you could do in a database, plus much more.

This notebook will outline many of the functions available in pandas. Generally, it will demonstrate some common or simpler usages of the function. You can always look at the documentation using help(pd.function) to get a full list of capabilities.

In [1]:
import pandas as pd

In [2]:
#pull_data
import get_data
get_data.get_file("cast.csv", "../data/")
get_data.get_file("release_dates.csv", "../data/")
get_data.get_file("titles.csv", "../data/")

True

## I/O and Basics 

We will import some data to play with - the IMDB database.

We read this in with the "from\_csv" command. There are many formats that pandas can accept directly - dictionaries, lists of lists, json, Excel, to name a few. Similarly, it is easy to output a DataFrame to any of these formats, using the .to_\* commands

Note that this is a relatively large data set (3.7M rows) but pandas will handle it reasonably well

In [3]:
%%time
cast = pd.DataFrame.from_csv('../data/cast.csv', index_col=None) 
print(cast.shape)

(3768347, 6)
CPU times: user 4.9 s, sys: 388 ms, total: 5.29 s
Wall time: 5.3 s


As demonstrated above, a DataFrame as a "shape" attribute, much like a numpy matrix. We can also call some functions on it, such as head() and tail() which return DataFrames which are subsets of the data.

In [4]:
cast.head(3)

,title,year,name,type,character,n
0,The Core,2003,Alejandro Abellan,actor,U.S.S. Soldier,NaN
1,Il momento di uccidere,1968,Remo De Angelis,actor,Dago,9.0
2,Across the Divide,1921,Thomas Delmar,actor,Dago,4.0


In [5]:
cast.tail(3)

,title,year,name,type,character,n
3768344,Para Para Para,2015,Birgul Yilmaz,actress,Guleser,NaN
3768345,Necroville,2007,Julian Wolf,actress,S&M Zombie,NaN
3768346,We Not Naughty,2012,Yun Xiang,actress,Bao's mother,NaN


It should be noted that almost all operations on a DataFrame return a new DataFrame and are not done in-place on the original object. So, unless we assign the result to a variable, there is no effect. 

So, if I want to keep the first 10 rows, I could do the following.

In [6]:
cast10 = cast.head(10)
print('Shape of cast is', cast.shape)
print('Shape of cast10 is', cast10.shape)

Shape of cast is (3768347, 6)
Shape of cast10 is (10, 6)


## Series, DataFrames and Indices

The building block of a DataFrame is the Series, which is essentially one column of the DataFrame. More specifically, it is a one-dimensional array with a particular type of data (e.g. int) and indexed. 

Indexes can also be of any type and need not be unique. If an index is not specified upon creation, the default is to simply use a numerical index from 0 to $n-1$, as seen in the DataFrame above, shown in the the unlabelled bolded column.

DataFrames are then a collection of these Series, joined on the index, resulting in two-dimensional data structure, with an index and labels for each Series to represent columns. The data types can vary across Series.

In [7]:
print(type(cast['year']))
cast['year'].head(5)

<class 'pandas.core.series.Series'>


0    2003
1    1968
2    1921
3    2012
4    1950
Name: year, dtype: int64

In [8]:
print(type(cast['type']))
cast['type'].head(5)

<class 'pandas.core.series.Series'>


0    actor
1    actor
2    actor
3    actor
4    actor
Name: type, dtype: object

In [9]:
#convert index to list to see first 5 elements
list(cast.index)[:5]

[0, 1, 2, 3, 4]

## Selection, Boolean Indexing and Setting

Here are some examples of common ways to subset data in pandas, similar to using a WHERE clause in SQL.

First, we can get a particular column, which gives us a Series. There are two ways to do this, either df['column_name'] or df.column_name as long as the column name is alphanumeric.

In [10]:
cast['type'].head(5)

0    actor
1    actor
2    actor
3    actor
4    actor
Name: type, dtype: object

In [11]:
cast.type.head(5)

0    actor
1    actor
2    actor
3    actor
4    actor
Name: type, dtype: object

We can also select rows and columns in various ways.

In [12]:
cast[3:10]

,title,year,name,type,character,n
3,Revan,2012,Diego James,actor,Dago,NaN
4,Un homme marche dans la ville,1950,Fabien Loris,actor,Dago,12.0
5,Malaria,1943,Michel Salina,actor,Dago,10.0
6,Ladies' Night,2003,Hugo Silva (III),actor,Dago,5.0
7,You Didn't Know,2014,Tony Smith (LXXVI),actor,Dago,NaN
8,Pecos è qui: prega e muori!,1967,Ignazio Spalla,actor,Dago,4.0
9,The Crop,2004,Vincent Stone,actor,Dago,5.0


In [13]:
cast[['title', 'year']].head(10)

,title,year
0,The Core,2003
1,Il momento di uccidere,1968
2,Across the Divide,1921
3,Revan,2012
4,Un homme marche dans la ville,1950
5,Malaria,1943
6,Ladies' Night,2003
7,You Didn't Know,2014
8,Pecos è qui: prega e muori!,1967
9,The Crop,2004


In [14]:
cast.iloc[3:5, 2:4]

,name,type
3,Diego James,actor
4,Fabien Loris,actor


In [15]:
cast.loc[3:5, ['name','type']] #as the documentation will tell you, unlike most python ranges, the end point value is included here

,name,type
3,Diego James,actor
4,Fabien Loris,actor
5,Michel Salina,actor


Now to get into some of the real power of pandas, we can filter rows in many complex ways.

It is useful to understand what is going on. The filtering is done by passing a Boolean Series to the DataFrame. This Boolean Series can be generated by applying operations to other Series in the DataFrame and can be combined in many ways. In particular:
* AND is given by &
* OR is given by |
* NOT is given by ~

Parentheses are quite important as these operators take precedence. A > 1 & B > 2 will be parsed as A > (1 & B) > 2 which will fail.

In [16]:
( cast.year > 2014 ).head() #this is the Boolean Series created by the criteria below

0    False
1    False
2    False
3    False
4    False
Name: year, dtype: bool

In [17]:
cast[cast.year >= 2014].head(10)

,title,year,name,type,character,n
7,You Didn't Know,2014,Tony Smith (LXXVI),actor,Dago,NaN
31,A Horse Story,2015,Josh Kopell,actor,Billy Wells,NaN
35,Northmen - A Viking Saga,2014,Johan Hegg,actor,Valli,NaN
48,Karthikeya,2014,Swathi Reddy,actress,Valli,NaN
49,Sathuranga Vettai,2014,Senthi,actress,Valli,9.0
77,Brotherhood,2014,John D. Green Sr.,actor,Blade,NaN
86,Random,2014,Mike Lepera,actor,Blade,NaN
95,The Chemist,2015,Sasha Mitchell,actor,Blade,NaN
122,Finding Oblivion,2014,Tyanna Bone,actress,Blade,67.0
131,Sannhetens løgn,2015,Daniel Bratteli,actor,Morten,NaN


In [18]:
cast[(cast.year >= 2014) & (cast.type == 'actress') & ~(cast.character == 'Orderly')].head(10)

,title,year,name,type,character,n
48,Karthikeya,2014,Swathi Reddy,actress,Valli,NaN
49,Sathuranga Vettai,2014,Senthi,actress,Valli,9.0
122,Finding Oblivion,2014,Tyanna Bone,actress,Blade,67.0
1723,KILD TV,2015,Haley Hussey,actress,Harvey,16.0
1724,Bee Sting,2014,Diva Nelson,actress,Harvey,5.0
1929,Q-4: Dream Corporation,2015,Erin Maxwell Albert,actress,Indigo,NaN
1930,Mr. Blue Shirt: The Inspiration,2015,Katherine Grant (V),actress,Indigo,NaN
1935,Dream in Indigo,2016,Shoniqua Shandai,actress,Indigo,NaN
2020,The Tailor of Autumn,2015,Heidi Yardley,actress,Oberon,NaN
2137,Ch/B,2014,Olga Dibtseva,actress,Correspondent,NaN


Generally, an Boolean Series or list that is the same length as the DataFrame (i.e. len(boolean_series) == df.shape[0]) can be passed as a filter. 

It is also possible to assign values based on filtering. Just like before, we use a Boolean Series to select rows and, usually, select a column and assign a replacement value.

The replacement value need not be constant, as long as it is the same size of the data it is replacing. 

This is similar to an UPDATE statement in SQL.

In [19]:
myfilter = (cast.year == 2014)
cast.loc[myfilter, 'year'] = 2014 #won't actually change the data in this case

In [20]:
cast.loc[myfilter, 'title'] = cast.title[myfilter] #again, won't change the data in this case

## Sorting and Dropping and Filling

Here are some common miscellaneous opertions that can be performed on a DataFrame.

We can first drop rows or columns, using the drop function. Notice the "axis" command, which is common in pandas. Many functions can operation on either rows or columns, and axis=0 and axis=1 specify these, respectively. axis=0 is the default and if we had not specified the axis, the command would have failed as there are no rows with an index value of 'title'.

In [21]:
cast.drop(['title', 'name'], axis=1).head(5)

,year,type,character,n
0,2003,actor,U.S.S. Soldier,NaN
1,1968,actor,Dago,9.0
2,1921,actor,Dago,4.0
3,2012,actor,Dago,NaN
4,1950,actor,Dago,12.0


Drop_duplicates will keep only unique rows, similar to a SELECT DISTINCT in SQL.

In [22]:
cast[['year']].drop_duplicates().head(10)

,year
0,2003
1,1968
2,1921
3,2012
4,1950
5,1943
7,2014
8,1967
9,2004
10,1977


We can also sort on column values. The by argument indicated the columns and the ascending argument is a Boolean or binary list indicating whether each column should be sorted ascending. 

There is an additional argument inplace which, if set to True, will do the sorting in place (i.e. will not return a new DataFrame)

In [23]:
cast.sort_values(by = ['year', 'title'], ascending = [1, 0]).head()

,title,year,name,type,character,n
244930,Miss Jerry,1894,William Courtenay,actor,Mr. Hamilton,NaN
2151447,Miss Jerry,1894,Chauncey Depew,actor,Himself - the Director of the New York Central...,NaN
3189637,Miss Jerry,1894,Blanche Bayliss,actress,Miss Geraldine Holbrook,NaN
498817,Can Can,1898,Mlle. Fraidora,actress,Herself,NaN
1087425,Jeffries-Sharkey Contest,1899,William A. Brady,actor,Himself,NaN


A very useful function is fillna which will replace all missing values with a specified value.

In [24]:
cast.fillna(-1).head()

,title,year,name,type,character,n
0,The Core,2003,Alejandro Abellan,actor,U.S.S. Soldier,-1.0
1,Il momento di uccidere,1968,Remo De Angelis,actor,Dago,9.0
2,Across the Divide,1921,Thomas Delmar,actor,Dago,4.0
3,Revan,2012,Diego James,actor,Dago,-1.0
4,Un homme marche dans la ville,1950,Fabien Loris,actor,Dago,12.0


This is similar to filtering and setting with pd.isnull which returns a Boolean series indicating whether values are missing.

In [25]:
cast.loc[pd.isnull(cast.n),['n']] = -1
cast.head(5)

,title,year,name,type,character,n
0,The Core,2003,Alejandro Abellan,actor,U.S.S. Soldier,-1.0
1,Il momento di uccidere,1968,Remo De Angelis,actor,Dago,9.0
2,Across the Divide,1921,Thomas Delmar,actor,Dago,4.0
3,Revan,2012,Diego James,actor,Dago,-1.0
4,Un homme marche dans la ville,1950,Fabien Loris,actor,Dago,12.0


## Information and Summarization 

There are two easy ways we can learn about a DataFrame:
* attributes - called by df.attribute which gives information about the DataFrame
* functions - called by df.function() which applies functions over the columns of a DataFrame. Or we can apply the function to a particular Series, df.column_name.function()



In [26]:
cast.columns

Index(['title', 'year', 'name', 'type', 'character', 'n'], dtype='object')

In [27]:
cast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3768347 entries, 0 to 3768346
Data columns (total 6 columns):
title        object
year         int64
name         object
type         object
character    object
n            float64
dtypes: float64(1), int64(1), object(4)
memory usage: 172.5+ MB


In [28]:
cast.count()

title        3768347
year         3768347
name         3768347
type         3768347
character    3768347
n            3768347
dtype: int64

In [29]:
cast.min()

title                    #5
year                   1894
name         !Mystery Girl!
type                  actor
character           !Nanseb
n                        -1
dtype: object

In [30]:
cast.max()

title        è solo questione di tempo
year                              2021
name                   Þórður Ólafsson
type                           actress
character                    üzletkötõ
n                                33613
dtype: object

In [31]:
cast.n.sum()

36224442.0

In [32]:
cast.year.mean()

1984.7142383649914

In [33]:
cast.year.median()

1995.0

In [34]:
cast.year.mode()

0    2014
dtype: int64

In [35]:
cast.describe()

,year,n
count,3.768347e+06,3.768347e+06
mean,1.984714e+03,9.612820e+00
std,2.808852e+01,2.518619e+01
min,1.894000e+03,-1.000000e+00
25%,1.963000e+03,-1.000000e+00
50%,1.995000e+03,4.000000e+00
75%,2.009000e+03,1.300000e+01
max,2.021000e+03,3.361300e+04


## Grouping

It is often useful to perform the above operations but to subsets of the data. For example, we might want to know the statistics of 'year' based on the actor/actress split. 

This is done in pandas using first using groupby. This produces a groupby object that can be treated similarly to a DataFrame, such as by selecting columns and applying functions to them. The computation is done by group. 

The result we get from the operation will have the grouped columns as index and the calculated values as columns of a DataFrame. If there is only one calculated value the result will be a Series. We can use reset_index() to bring the index back as columns. 

Of course, this is similar to GROUP BY in SQL.

In [36]:
cast.groupby('type')['year'].median()

type
actor      1993
actress    1998
Name: year, dtype: int64

In [37]:
cast.groupby('type')['year'].median().reset_index()

,type,year
0,actor,1993
1,actress,1998


In [38]:
cast.groupby('name').size().sort_values(ascending=0).head(10) #count number of records per actor and sort; note we are sorting a Series so don't specify a column

name
Bess Flowers       807
Herman Hack        663
Sam Harris (II)    621
Lee Phelps         612
Frank O'Connor     600
Harold Miller      572
Tom London         563
Frank Ellis        539
Larry Steers       535
Franklyn Farnum    531
dtype: int64

We can also write our own custom functions. lambda functions are incredibly useful here. 

In [39]:
cast.groupby('type').year.agg({'min_year_no_1800': lambda x: min([a for a in x if a >=1900])}) 

,min_year_no_1800
type,
actor,1900
actress,1905


## String and Date Operations

There are two sets of perations that are useful when columns are strings or dates:
* df.string_column.str.\* which are functions to call on the string
* df.date_column.dt.\* which are attributes of the date that can be accessed

We will read in data with some date info to demonstrate these.

In [40]:
release_dates = pd.read_csv('../data/release_dates.csv', index_col=None)
release_dates.head()

,title,year,country,date
0,0_1_0,2008.0,Poland,2008-11-14 00:00:00
1,Ai no Sanka,1967.0,Japan,1967-01-01 00:00:00
2,A Thousand to One,1920.0,USA,1920-12-07 00:00:00
3,A Prince of a King,1923.0,USA,1923-10-13 00:00:00
4,A Prince of a King,1923.0,Netherlands,1924-08-08 00:00:00


In [41]:
release_dates.date = pd.to_datetime(release_dates.date) #convert to date

In [42]:
release_dates.date.dt.dayofyear.head()

0    319
1      1
2    342
3    286
4    221
Name: date, dtype: int64

In [43]:
release_dates.date.dt.month.head()

0    11
1     1
2    12
3    10
4     8
Name: date, dtype: int64

In [44]:
release_dates.country[release_dates.country.str.contains("land")].head(5)

0          Poland
4     Netherlands
7     Netherlands
18    Netherlands
25        Finland
Name: country, dtype: object

In [45]:
release_dates.country[release_dates.country.str.startswith("J")].head(5)

1     Japan
6     Japan
15    Japan
41    Japan
54    Japan
Name: country, dtype: object

In [46]:
release_dates.title.str.extract('^([^ ]+)', expand=True).head(5) #get first word

,0
0,0_1_0
1,Ai
2,A
3,A
4,A


## Merging

Multiple DataFrames can be merged together, similar to a JOIN clause in SQL. There are many options in joining, such as what to join on, whether to do an inner/outer join, how to rename columns.

The simplest case is doing an inner join on all columns in common.

In [47]:
cast.head(5).merge(release_dates) #note this join results in more than 5 rows since each movie will have multiple release date entries

,title,year,name,type,character,n,country,date
0,The Core,2003,Alejandro Abellan,actor,U.S.S. Soldier,-1.0,Canada,2003-03-28
1,The Core,2003,Alejandro Abellan,actor,U.S.S. Soldier,-1.0,Ireland,2003-03-28
2,The Core,2003,Alejandro Abellan,actor,U.S.S. Soldier,-1.0,Mexico,2003-03-28
3,The Core,2003,Alejandro Abellan,actor,U.S.S. Soldier,-1.0,Spain,2003-03-28
4,The Core,2003,Alejandro Abellan,actor,U.S.S. Soldier,-1.0,UK,2003-03-28
5,The Core,2003,Alejandro Abellan,actor,U.S.S. Soldier,-1.0,USA,2003-03-28
6,The Core,2003,Alejandro Abellan,actor,U.S.S. Soldier,-1.0,Philippines,2003-04-02
7,The Core,2003,Alejandro Abellan,actor,U.S.S. Soldier,-1.0,Argentina,2003-04-03
8,The Core,2003,Alejandro Abellan,actor,U.S.S. Soldier,-1.0,Germany,2003-04-03
9,The Core,2003,Alejandro Abellan,actor,U.S.S. Soldier,-1.0,Hong Kong,2003-04-03


## Using Pandas to Answer Questions

We will give some examples of using the three data sets (titles, cast, release_dates) to answer some questions using pandas

In [48]:
%%time
titles = pd.DataFrame.from_csv('../data/titles.csv', index_col=None)
print(titles.shape)

(306705, 2)
CPU times: user 264 ms, sys: 3.95 ms, total: 268 ms
Wall time: 266 ms


In [49]:
# 1. How many movies are listed in the `titles` dataframe?
len(titles)

306705

In [50]:
# 2. What is the name and year of the very first movie ever made?
titles.sort_values('year').head(1)

,title,year
172161,Miss Jerry,1894.0


In [51]:
# 3. How many years into the future does the IMDB database list movie titles?
titles.sort_values('year').tail(3)#.year - 2015

,title,year
284322,The Tree of Death,2022.0
52606,Crime and Punishment,2022.0
170430,New in Village,2024.0


In [52]:
# 4. How many movies listed in `titles` came out in 1950?

len(titles[titles.year == 1950])

1690

In [53]:
# 5. How many movies came out in each year of the 1970s?

for y in range(1970, 1980):
    print(y, (titles.year == y).sum()) #summing a Boolean Series gives the number that are True

1970 3317
1971 3145
1972 3021
1973 2901
1974 2860
1975 2834
1976 2835
1977 2810
1978 2894
1979 3021


In [54]:
# 6. Use "groupby" to determine how many movies came out in each year of the 1970s.
titles.groupby('year').size().loc[1970:1979]

year
1970.0    3317
1971.0    3145
1972.0    3021
1973.0    2901
1974.0    2860
1975.0    2834
1976.0    2835
1977.0    2810
1978.0    2894
1979.0    3021
dtype: int64

In [55]:
# 7. What are the names of the movies made through 1906?

titles[titles.year <= 1906][['title']]

,title
2863,Abajo la careta
4010,After Dark in Central Park
7873,A Very Laughable Mixup
11843,Boarding School Girls' Pajama Parade
17859,Alibaba and the Forty Thieves
17991,Around the Big Curves on the Manhattan Elevate...
18043,Bombardment of Matanzas
19506,Battle of Jeffries and Sharkey for Championshi...
23718,Bettlerstolz
23987,Bohemios


In [56]:
# 8. What movies have titles that fall between Star Trek and Star Wars in the alphabet?

titles[(titles.title >= 'Star Trek') & (titles.title <= 'Star Wars')].sort_values('title')

,title,year
230886,Star Trek,2009.0
216815,Star Trek I: Specter of the Past,2010.0
220793,Star Trek III: The Search for Spock,1984.0
228520,Star Trek IV: The Voyage Home,1986.0
241619,Star Trek Into Darkness,2013.0
235350,Star Trek V: The Final Frontier,1989.0
224302,Star Trek VI: The Undiscovered Country,1991.0
226252,Star Trek World Tour,1998.0
233186,Star Trek: First Contact,1996.0
239229,Star Trek: Generations,1994.0


In [57]:
# 9. What are the 15 most common movie titles in film history?

titles.title.value_counts().head(15) #value_counts is a quick way to do a groupby, size and sort

Carmen                  20
Hamlet                  19
Anna Karenina           16
Macbeth                 14
Kismet                  14
The Three Musketeers    13
Anna                    13
Maya                    13
Underground             12
Eva                     12
Temptation              12
Blood Money             12
Jackpot                 11
The Outsider            11
Vendetta                11
Name: title, dtype: int64

In [58]:
# 10. What are the 5 longest movie titles ever?

pd.set_option('max_colwidth', 300) #for printing/display purposes

t = titles.copy() #so we don't change the original
t['len'] = t.title.str.len()
t = t.sort('len', ascending=False)
t.head()

/mypath/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,title,year,len
184221,"Night of the Day of the Dawn of the Son of the Bride of the Return of the Revenge of the Terror of the Attack of the Evil Mutant Hellbound Flesh Eating Crawling Alien Zombified Subhumanoid Living Dead, Part 5",2011.0,208
178896,"Night of the Day of the Dawn of the Son of the Bride of the Return of the Revenge of the Terror of the Attack of the Evil, Mutant, Hellbound, Flesh-Eating Subhumanoid Zombified Living Dead, Part 3",2005.0,196
138326,Las poquianchis (De los pormenores y otros sucedidos del dominio público que acontecieron a las hermanas de triste memoria a quienes la maledicencia así las bautizó),1976.0,165
67489,Entrei em Pânico ao Saber o que Vocês Fizeram na Sexta-feira 13 do Verão Passado Parte 2 - A Hora da Volta da Vingança dos Jogos Mortais de Halloween,2011.0,149
12366,Ang kaisa-isang konsiyerto ng kagila-gilalas na kombo ni Kumander Kulas at ng kanyang kawawang kalabaw sa walang katapusang kalsada ng Kamyas,2011.0,141


In [59]:
# 11. What are the 15 most popular movie titles, if you strip off the suffixes like
#    (II) and (III) that the IMDB adds to distinguish movies shown in the same year?

titles.title.str.extract('^([^(]*)').value_counts().head(15)

/mypath/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


                        34
Carmen                  20
Hamlet                  19
Broken                  16
Anna Karenina           16
Macbeth                 14
Kismet                  14
The Three Musketeers    13
Anna                    13
Maya                    13
Eva                     12
Underground             12
Temptation              12
Love                    12
Blood Money             12
Name: title, dtype: int64

In [60]:
# 13. In how many of his movies was Sidney Poitier the lead (`n==1`)?

len(cast[(cast.name == 'Sidney Poitier') & (cast.n==1)])

21

In [61]:
# 14. Who was credited in the 1972 version of Sleuth, in order by `n` rank?

cast[(cast.title == 'Sleuth') & (cast.year==1972)].sort_values('n')

,title,year,name,type,character,n
2814149,Sleuth,1972,Laurence Olivier,actor,Andrew Wyke,1.0
1984265,Sleuth,1972,Michael Caine,actor,Milo Tindle,2.0
2003340,Sleuth,1972,Alec Cawthorne,actor,Inspector Doppler,3.0
2705846,Sleuth,1972,John Matthews (II),actor,Detective Sergeant Tarrant,4.0
3291632,Sleuth,1972,Eve Channing (III),actress,Marguerite Wyke,5.0
2692692,Sleuth,1972,Teddy Martin,actor,Police Constable Higgs,6.0


In [62]:
# 15. What are the 11 most common character names in movie history?

cast.character.value_counts().head(11)

Himself      24293
Dancer       11697
Extra         8865
Reporter      7792
Doctor        7666
Herself       7500
Policeman     7245
Student       6694
Nurse         6635
Bartender     6298
Zombie        5785
Name: character, dtype: int64

In [63]:
# 16. Which actors have played the role “Waiter” the most times?

cast[cast.character == 'Waiter'].name.value_counts().head(10)

William H. O'Brien    80
Jack Chefe            52
Gino Corrado          33
George Davis          28
Alexander Pollard     22
Bobby Barber          20
Leo White             20
Albert Pollet         19
Thomas Martin         18
Alphonse Martell      15
Name: name, dtype: int64

In [64]:
# 18a. Create a DataFrame with the 50 most common character names in film.
chars_top50 = cast.character.value_counts().head(50).reset_index()
chars_top50.columns = ['character', 'character_ct']
chars_top50.head()

,character,character_ct
0,Himself,24293
1,Dancer,11697
2,Extra,8865
3,Reporter,7792
4,Doctor,7666


In [65]:
#18b  Which are most often played by men?
chars_by_type = chars_top50.merge(cast).groupby(['character', 'type']).size()
chars_by_type = chars_by_type .unstack() #unstack will take indices with multiple levels an unstack them
chars_by_type['male_ratio'] = chars_by_type.actor / (chars_by_type.actor + chars_by_type.actress)
chars_by_type = chars_by_type.sort_values('male_ratio', ascending=0)
chars_by_type.head(5)

type,actor,actress,male_ratio
character,,,
Townsman,4963,3,0.999396
John,2628,2,0.999240
Henchman,4876,4,0.999180
Policeman,7233,12,0.998344
Himself,24251,42,0.998271


In [66]:
#18c  Which are most often played by men?
chars_by_type.tail(5)

type,actor,actress,male_ratio
character,,,
Anna,86,2868,0.029113
Mary,46,2232,0.020193
Girl,43,2512,0.016830
Maria,27,3080,0.008690
Herself,59,7441,0.007867


In [67]:
# 18d. …which of those 50 characters have a ratio closest to 0.5?
chars_by_type[(chars_by_type.male_ratio > 0.4) & (chars_by_type.male_ratio < 0.6)]

type,actor,actress,male_ratio
character,,,
Student,3845,2849,0.574395
Singer,1706,1811,0.485073
Teacher,1079,1234,0.466494


### Who has been in the most movies

In [68]:
# 19. Which actors or actresses appeared in the most movies in the year 1975?
cast[cast.year == 1975].name.value_counts().head(10)

Nestore Cavaricci       14
Akira Takahashi (II)    13
Tom Felleghy            13
Timuçin Caymaz          13
Dharmendra              12
Naomi Oka               12
Aruna Irani             12
Femi Benussi            11
Helen                   11
Asrani                  11
Name: name, dtype: int64

In [69]:
%%time
# 20. Create a `cast_by_title_year` dataframe indexed by title and year
#    to use in the next few questions.

#indexing can speed up opertions in which we will often filter or sort by the indexed columns

cast_by_title_year = cast.set_index(['title', 'year']).sort_index()
cast_by_title_year.head()

CPU times: user 12.3 s, sys: 751 ms, total: 13.1 s
Wall time: 13.1 s


In [70]:
%%time
# 22. Write a `for` loop that, for the top 9 actors in the 1977 movie Star Wars,
#    determines how many movies they starred in after 1977.

names = cast_by_title_year[cast_by_title_year.n > -1].loc['Star Wars',1977].sort_values('n').head(9).name
for name in names:
    print(name, len(cast[(cast.name == name) & (cast.year > 1977)]))

Mark Hamill 42
Harrison Ford 44
Carrie Fisher 40
Peter Cushing 10
Alec Guinness 9
Anthony Daniels 11
Kenny Baker 18
Peter Mayhew (II) 7
David Prowse 5
CPU times: user 4.99 s, sys: 55.1 ms, total: 5.05 s
Wall time: 5.03 s


In [71]:
%%time
# 23. Create an indexed version of `cast` that, once built, lets you answer
#    the previous question with a `for` loop that finishes in under a second.

i = cast.set_index('name').sort_index()

CPU times: user 19.4 s, sys: 311 ms, total: 19.7 s
Wall time: 19.7 s


In [72]:
%%time
for name in names:
    c = i.loc[name]
    c = c[c.year > 1977]
    #c = c[(c.character != 'Himself') & (c.character != 'Herself')]
    print(name, len(c))

Mark Hamill 42
Harrison Ford 44
Carrie Fisher 40
Peter Cushing 10
Alec Guinness 9
Anthony Daniels 11
Kenny Baker 18
Peter Mayhew (II) 7
David Prowse 5
CPU times: user 2.1 s, sys: 23.3 ms, total: 2.12 s
Wall time: 2.13 s


In [73]:
# 24. How many people were cast in each of the movies named "Hamlet”?

cast[cast.title == 'Hamlet'].groupby('year').size() 

year
1910     1
1911     6
1913    21
1921     9
1948    25
1954     1
1964    33
1969    24
1973     8
1976    11
1987     3
1990    29
1996    55
2000    38
2009    17
2011    12
2015     6
dtype: int64

In [74]:
# 25a. Define “leading actor” as an actor or actress whose `n==1`
#    and “supporting actor” as `n==2` — what is the average year
#    of all the supporting roles Judi Dench has had?

cast[(cast.name == 'Judi Dench') & (cast.n==2)].year.median()

1986.5

In [75]:
# 25b. What is the average year of Judi Dench’s leading roles —
#    is her career moving forwards toward leading roles
#    or backwards towards supporting ones?

cast[(cast.name == 'Judi Dench') & (cast.n==1)].year.median()

2005.5

In [76]:
# 26. In which months is a movie whose name ends in the word "Christmas"
#    most likely to be released?

release_dates[(release_dates.title.str.endswith('Christmas')) & (release_dates.country == 'USA')].date.dt.month.value_counts()

11    19
12    11
10     9
1      2
7      1
4      1
2      1
Name: date, dtype: int64